In [209]:
# import numpy as np
import pandas as pd
import string
# from collections import Counter
from sklearn.model_selection import train_test_split

In [249]:
validated = pd.read_csv('/media/ks/Elements/cv-corpus-13.0-2023-03-09/sw/validated.tsv', sep='\t', low_memory=False)
reported = pd.read_csv('/media/ks/Elements/cv-corpus-13.0-2023-03-09/sw/reported.tsv', sep='\t')
other = pd.read_csv('/media/ks/Elements/cv-corpus-13.0-2023-03-09/sw/other.tsv', sep='\t', low_memory=False)
invalidated = pd.read_csv('/media/ks/Elements/cv-corpus-13.0-2023-03-09/sw/invalidated.tsv', sep='\t')

In [4]:
# total number of unique sentences 
len(set(pd.concat([validated['sentence'], reported['sentence'], other['sentence'], invalidated['sentence']])))

86533

In [5]:
# total number of clips
len(pd.concat([validated['sentence'], reported['sentence'], other['sentence'], invalidated['sentence']]))

621716

Text normalization function
1. we lower all letters
2. we get rid of leading and trailing spaces
3. we get rid of extra spaces
4. remove punctuation
5. dropping all the sentences which have digits, these look quite problematic!!
6. remove all characters not in my 'allowed' list

In [260]:
def normalize_text(df):
    df.loc[:,'sentence'] = df['sentence'].transform(lambda s: s.lower())
    df.loc[:,'sentence'] = df['sentence'].transform(lambda s: s.strip())
    df.loc[:,'sentence'] = df['sentence'].transform(lambda s: " ".join(s.split()))
    df.loc[:,'sentence'] = df['sentence'].transform(lambda s: s.translate(str.maketrans('', '', string.punctuation)))
    df = df[~df['sentence'].str.contains('\d')]
    characters_allowed = [' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 
                          'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z',]
    df.loc[:,'sentence'] = df['sentence'].transform(lambda s: ''.join(char for char in s if char in characters_allowed or char == ' '))

    return df


In [261]:
normalize_text(validated)

,client_id,path,sentence,up_votes,down_votes,age,gender,accents,variant,locale,segment
0,0133d8ddf5c1a3c678fde017e0b07d2835bfd707d5b3ec...,common_voice_sw_31428161.mp3,wachambuzi wa soka wanamtaja messi kama nyota ...,2,0,twenties,female,NaN,NaN,sw,NaN
1,01c95772efd3fbe4a1122206c7474c77ed6591c8c9fb00...,common_voice_sw_30317714.mp3,romario aliingia kwenye orodha ya wachezaji wa...,2,1,NaN,NaN,NaN,NaN,sw,NaN
2,023711185d4404ff398c2697f2e72868d1ecf69a92b581...,common_voice_sw_32116997.mp3,sote twesangaa twelipomuona mwalimu ali apika,2,1,twenties,male,NaN,NaN,sw,NaN
3,0244639ffd7ec755a01b21ea204735ca3c44443e9cf46c...,common_voice_sw_29002392.mp3,inajulikana kama shina la warangi,2,0,NaN,NaN,NaN,NaN,sw,NaN
4,04e78dc3038488a080fe3c76c28602d0db9e4eec2efbf0...,common_voice_sw_37088105.mp3,ni kikwazo kikubwa katika maendeleo ya afrika,8,5,NaN,NaN,NaN,NaN,sw,NaN
...,...,...,...,...,...,...,...,...,...,...,...
231137,457b3a2570720101c75d297cde767487e8f0a1a7f714cb...,common_voice_sw_30475794.mp3,hana pesa za kutosha,2,0,thirties,male,NaN,NaN,sw,NaN
231138,457b3a2570720101c75d297cde767487e8f0a1a7f714cb...,common_voice_sw_30475795.mp3,chakula chako si kitamu vile,2,1,thirties,male,NaN,NaN,sw,NaN
231139,457b3a2570720101c75d297cde767487e8f0a1a7f714cb...,common_voice_sw_30475797.mp3,aliitwa mara tatu,2,0,thirties,male,NaN,NaN,sw,NaN
231140,457b3a2570720101c75d297cde767487e8f0a1a7f714cb...,common_voice_sw_30475802.mp3,maradhi ya tumbo sio mchezo,2,1,thirties,male,NaN,NaN,sw,NaN


In [250]:
validated['sentence'] = validated['sentence'].transform(lambda s: s.lower())
validated['sentence'] = validated['sentence'].transform(lambda s: s.strip())
validated['sentence'] = validated['sentence'].transform(lambda s: " ".join(s.split()))
validated['sentence'] = validated['sentence'].transform(lambda s: s.translate(str.maketrans('', '', string.punctuation)))
validated = validated[~validated['sentence'].str.contains('\d')]
characters_allowed = [' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 
                      'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z',]
validated['sentence'] = validated['sentence'].transform(lambda s: ''.join(char for char in s if char in characters_allowed or char == ' '))

In [6]:
# total number of speakers
len(set(pd.concat([validated['client_id'], other['client_id'], invalidated['client_id']])))

1103

In [7]:
# filtering out data from the following PRs
# https://github.com/common-voice/common-voice/pull/3957 by Badili innovations, livestock data
# https://github.com/common-voice/common-voice/pull/3970 by Badili innovations, livestock data
# https://github.com/common-voice/common-voice/pull/3640 by Kat, Dialect/Variant data

In [251]:
kiunguja = pd.read_csv('sampla_data/sw-kiunguja.txt', header=None)
baratz = pd.read_csv('sampla_data/sw-baratz.txt', header=None)
kibajuni = pd.read_csv('sampla_data/sw-kibajuni.txt', header=None)
kimakunduchi = pd.read_csv('sampla_data/sw-kimakunduchi.txt', header=None)
kimvita = pd.read_csv('sampla_data/sw-kimvita.txt', header=None)
kipemba = pd.read_csv('sampla_data/sw-kipemba.txt', header=None)
kitumbatu = pd.read_csv('sampla_data/sw-kitumbatu.txt', header=None)

In [9]:
filter_out = pd.concat([kiunguja[0],baratz[0],kibajuni[0],kimakunduchi[0],kimvita[0],kipemba[0],kitumbatu[0]], ignore_index=True)
filter_out = pd.DataFrame(filter_out)

In [10]:
# finding the instances from the filter_out set
filter_sets = []
for idx, value in validated['sentence'].items():
    for idx2, value2 in filter_out[0].items():
        if value == value2:
            filter_sets.append(idx)

In [11]:
# filtering out the instances in out filter set
filtered_validated = validated[~validated.index.isin(filter_sets)]
len(filtered_validated)

225454

In [252]:
# function to take out our dialect/variant evaluation sets
def dialect_evaluation_sets(df):
    dialect_set = []
    for idx, value in validated['sentence'].items():
        for idx2, value2 in df[0].items():
            if value == value2:
                dialect_set.append(idx)
            
    return validated[validated.index.isin(dialect_set)]

In [ ]:
dialect_evaluation_sets(kiunguja).to_csv('sampla_data/eval_kiunguja.csv')
dialect_evaluation_sets(kibajuni).to_csv('sampla_data/eval_kibajuni.csv')
dialect_evaluation_sets(baratz).to_csv('sampla_data/eval_baratz.csv')
dialect_evaluation_sets(kimakunduchi).to_csv('sampla_data/eval_kimakunduchi.csv')
dialect_evaluation_sets(kimvita).to_csv('sampla_data/eval_kimvita.csv')
dialect_evaluation_sets(kipemba).to_csv('sampla_data/eval_kipemba.csv')
dialect_evaluation_sets(kitumbatu).to_csv('sampla_data/eval_kitumbatu.csv')

In [12]:
# considerations in creating the train, dev, test sets
# repeated instances of sentences should only be in one set
# repeated audios contributed by a single speaker should also all only be in one set
# balancing gender in the various sets
# we also drop duplicates where a user may have contributed to an individual sentence more than once (drops 5742) 
# leaves us with 219712
len(set(filtered_validated['client_id']))

853

In [121]:
# finding the number of contributions per speaker
# working towards 60:20:20
# contributors 1-6,11-16: 27908 + 9995 + 8593 + 8179 + 8029 + 6055 + 4606 + 4045 + 3911 + 3810 + 3703 + 3428 = 92262 (train)
# contributors 7-8,17-18: 5589 + 4759 + 3391 + 3372 = 17111(dev)
# contributors 9-10,19-20: 4701 + 4667 + 3350 + 3332 = 16050 (test)
# contributions_count = 
# list(Counter(list(filtered_validated['client_id'])).items())
# contributions_count
Counter(list(filtered_validated['client_id'])).most_common()

[('457b3a2570720101c75d297cde767487e8f0a1a7f714cbe563bf8a0216269e3580b1f8601e37ca172516a0858c306c2d25a18b1fff3a38a80452d94baecabcb6',
  27908),
 ('f615bda60b0fe910daf2d6c8962f4a3eb317d0588c5d99cfb2bb8faafb1c747ae00b8d30e7c616bec3b350579021de29fbf7040017cff69c73209120a28a45c5',
  9995),
 ('6c314661d4c610349387794fe95b0fcd6943919a861922cd2468c55247f805bad17b67f9007c2f196b27866355791bd3db21d64de93d97c6abb0660e2b8455f7',
  8593),
 ('f801f0071af5336c89e558dc35188c89be8f0c0c89f508e9d49f28feebc1a49bac9b3a6c6f4c0537be094d833ea7a236ff08f1bc7a79dc4d3d0d96913a6e15d4',
  8179),
 ('069cff5d5f20907814a9c1cc6df8c72ff1d2680a27c49a7fe2fe806fa46afce61013f266d381402a59060c02e2956209c9e5cb8004bd900b624199b780e7c626',
  8029),
 ('66858b5ce4429ee1297e082789724acb02c7c3d38e543b03815604906fa731232f78b0a5b822b036f3d856b27f3815e1837349b842f8aa6e0be7ab68940e39be',
  6055),
 ('86df220eb7e675996eb365baa70b06d2a41d91aaaf4a8e55be53c861ff42cb0d6eeca5ca906d75436ef122bbd0815caab96cf6316b01ab962c9e9814e4dd6f28',
  5589)

In [46]:
train = contributions_count[:int(len(contributions_count)* 0.6)]
# dev = contributions_count[:int(len(contributions_count)* 0.6)]
# test = contributions_count[:int(len(contributions_count)* 0.6)]

In [48]:
sum([tup[1] for tup in train])
# train

3790

ChatGPT function

In [77]:
import pandas as pd

def split_data(df):
    # Group by sentence and client_id
    grouped = df.groupby(['sentence', 'client_id'], as_index=False)
    
    # Get the first index of each group
    idxs = grouped.first().index
    
    # Split indices into train, validation, and test sets
    train_idxs, val_test_idxs = train_test_split(idxs, test_size=0.4, random_state=42)
    val_idxs, test_idxs = train_test_split(val_test_idxs, test_size=0.5, random_state=42)
    
    # Get the corresponding sentences and client_ids for each set of indices
    train = df.loc[df.index.isin(train_idxs)]
    val = df.loc[df.index.isin(val_idxs)]
    test = df.loc[df.index.isin(test_idxs)]
    
    return train, val, test


In [78]:
train, val, test = split_data(filtered_validated)

In [27]:
test.iloc[0]['client_id']

'0c56d038f98bd61cfda63020a1c87f65f1aee9a68560b3ef7731f8f40d1b253951956e75f0a9567be639087d27b00f25032ecc1c3e2e6831a440b74bc2783ee4'

In [47]:
'0c56d038f98bd61cfda63020a1c87f65f1aee9a68560b3ef7731f8f40d1b253951956e75f0a9567be639087d27b00f25032ecc1c3e2e6831a440b74bc2783ee4' in val['client_id'].unique()

False

In [34]:
train

,client_id,path,sentence,up_votes,down_votes,age,gender,accents,variant,locale,segment
1,01c95772efd3fbe4a1122206c7474c77ed6591c8c9fb00...,common_voice_sw_30317714.mp3,romario aliingia kwenye orodha ya wachezaji wa...,2,1,NaN,NaN,NaN,NaN,sw,NaN
3,0244639ffd7ec755a01b21ea204735ca3c44443e9cf46c...,common_voice_sw_29002392.mp3,Inajulikana kama shina la Warangi.,2,0,NaN,NaN,NaN,NaN,sw,NaN
5,0627e3da13c92a346dc119f4939b6d84d4712093e75656...,common_voice_sw_37214113.mp3,alikamatwa katika wilaya ya mkono kwa sasa,7,2,twenties,NaN,NaN,NaN,sw,NaN
7,0c9c9bd95bcc240196ce39556998679def64839e04be70...,common_voice_sw_37205423.mp3,mbele ya sanamu tukufu ya Kaisari,2,0,twenties,male,NaN,NaN,sw,NaN
8,1985f698c3e98c04967ee93e5c12c6ff1b82349022d281...,common_voice_sw_35774930.mp3,katika mambo yote Hivyo polepole wenyeji walia...,4,0,NaN,NaN,NaN,NaN,sw,NaN
...,...,...,...,...,...,...,...,...,...,...,...
225402,457b3a2570720101c75d297cde767487e8f0a1a7f714cb...,common_voice_sw_30334706.mp3,Wanaoingia nchini kwa mara ya kwanza baada ya ...,2,0,thirties,male,NaN,NaN,sw,NaN
225403,457b3a2570720101c75d297cde767487e8f0a1a7f714cb...,common_voice_sw_30334707.mp3,Baadhi ya mifumo iliundwa ili kukazia na kuong...,2,0,thirties,male,NaN,NaN,sw,NaN
225404,457b3a2570720101c75d297cde767487e8f0a1a7f714cb...,common_voice_sw_30334709.mp3,Miaka iliyofuata kukawepo fujo na machafuko ku...,3,2,thirties,male,NaN,NaN,sw,NaN
225407,457b3a2570720101c75d297cde767487e8f0a1a7f714cb...,common_voice_sw_30334717.mp3,watalii wanaotaka kitumia magari binafsi ngoro...,3,2,thirties,male,NaN,NaN,sw,NaN


In [50]:
filtered_validated['gender'].value_counts()

male      88782
female    65840
other        80
Name: gender, dtype: int64

In [53]:
filtered_validated['age'].value_counts()

twenties    88139
thirties    38812
fifties     11178
fourties    10925
sixties       858
teens         143
Name: age, dtype: int64

In [54]:
test['age'].value_counts()

twenties    17690
thirties     6446
fifties      2291
fourties     2135
sixties       178
teens          33
Name: age, dtype: int64

In [55]:
val['age'].value_counts()

twenties    17511
thirties     6771
fifties      2215
fourties     2200
sixties       149
teens          35
Name: age, dtype: int64

In [56]:
train['age'].value_counts()

twenties    52938
thirties    19862
fifties      6672
fourties     6590
sixties       531
teens          75
Name: age, dtype: int64

In [57]:
filtered_validated.groupby(['gender', 'client_id'], as_index=False)

In [138]:
import pandas as pd
from sklearn.model_selection import train_test_split

def split_data(df):
    # Group by sentence and client_id
    grouped = df.groupby(['sentence', 'client_id'], as_index=False)
    
    # Get the first index of each group
    idxs = grouped.first().index
    
    # Split indices into train, validation, and test sets
    train_idxs, val_test_idxs = train_test_split(idxs, test_size=0.4, random_state=42)
    val_idxs, test_idxs = train_test_split(val_test_idxs, test_size=0.75, random_state=42)
    val2_idxs, test2_idxs = train_test_split(test_idxs, test_size=0.5, random_state=42)
    
    # Get the corresponding sentences and client_ids for each set of indices
    train = df.loc[df.index.isin(train_idxs)]
    val = df.loc[df.index.isin(val_idxs)]
    val2 = df.loc[df.index.isin(val2_idxs)]
    test = df.loc[df.index.isin(test2_idxs)]
    
    return train, val, val2, test


In [147]:
train, eval0, dev, test = split_data(filtered_validated)

In [148]:
len(train)

131833

In [149]:
len(dev)

32974

In [150]:
len(test)

32990

In [151]:
len(eval0)

21924

In [116]:
train.iloc[3]['client_id']

'0627e3da13c92a346dc119f4939b6d84d4712093e75656f571427c8e965b8c67515ba14975b37f3901553d25fac7f00beed1f382b4a1a65fd5e60dc4329069f3'

In [119]:
'0627e3da13c92a346dc119f4939b6d84d4712093e75656f571427c8e965b8c67515ba14975b37f3901553d25fac7f00beed1f382b4a1a65fd5e60dc4329069f3' in val['client_id'].unique()

False

In [123]:
Counter(list(val['client_id'])).most_common()

[('457b3a2570720101c75d297cde767487e8f0a1a7f714cbe563bf8a0216269e3580b1f8601e37ca172516a0858c306c2d25a18b1fff3a38a80452d94baecabcb6',
  2750),
 ('f615bda60b0fe910daf2d6c8962f4a3eb317d0588c5d99cfb2bb8faafb1c747ae00b8d30e7c616bec3b350579021de29fbf7040017cff69c73209120a28a45c5',
  1256),
 ('f801f0071af5336c89e558dc35188c89be8f0c0c89f508e9d49f28feebc1a49bac9b3a6c6f4c0537be094d833ea7a236ff08f1bc7a79dc4d3d0d96913a6e15d4',
  1057),
 ('6c314661d4c610349387794fe95b0fcd6943919a861922cd2468c55247f805bad17b67f9007c2f196b27866355791bd3db21d64de93d97c6abb0660e2b8455f7',
  1049),
 ('069cff5d5f20907814a9c1cc6df8c72ff1d2680a27c49a7fe2fe806fa46afce61013f266d381402a59060c02e2956209c9e5cb8004bd900b624199b780e7c626',
  997),
 ('66858b5ce4429ee1297e082789724acb02c7c3d38e543b03815604906fa731232f78b0a5b822b036f3d856b27f3815e1837349b842f8aa6e0be7ab68940e39be',
  764),
 ('86df220eb7e675996eb365baa70b06d2a41d91aaaf4a8e55be53c861ff42cb0d6eeca5ca906d75436ef122bbd0815caab96cf6316b01ab962c9e9814e4dd6f28',
  704),
 (

In [172]:
eval0['age'].value_counts()

twenties    8728
thirties    3410
fourties    1139
fifties     1068
sixties       74
teens         20
Name: age, dtype: int64

In [153]:
train.to_csv('sampla_data/train.csv')
dev.to_csv('sampla_data/dev.csv')
test.to_csv('sampla_data/test.csv')

In [185]:
eval0[eval0['gender'] == 'male'].head(5000).to_csv('sampla_data/eval_male.csv')
eval0[eval0['gender'] == 'female'].head(5000).to_csv('sampla_data/eval_female.csv')
eval0[eval0['age'] == 'twenties'].head(5000).to_csv('sampla_data/eval_20s.csv')

In [186]:
pd.concat([eval0[eval0['age'] == 'thirties'].head(2719), eval0[eval0['age'] == 'fourties'], eval0[eval0['age'] == 'fifties'], eval0[eval0['age'] == 'sixties']]).to_csv('sampla_data/eval_o30s.csv')

In [188]:
eval0.loc[(eval0['gender'] == 'male') & (eval0['age'] == 'twenties')].to_csv('sampla_data/eval_20sMale.csv')
eval0.loc[(eval0['gender'] == 'female') & (eval0['age'] == 'twenties')].to_csv('sampla_data/eval_20sFemale.csv')

In [189]:
eval0.loc[(eval0['gender'] == 'female') & ~(eval0['age'] == 'twenties') & ~(eval0['age'] == 'teens')].to_csv('sampla_data/eval_o30sFemale.csv')
eval0.loc[(eval0['gender'] == 'male') & ~(eval0['age'] == 'twenties') & ~(eval0['age'] == 'teens')].to_csv('sampla_data/eval_o30sMale.csv')

In [191]:
# finding the instances
kiunguja_sets = []
for idx, value in validated['sentence'].items():
    for idx2, value2 in kiunguja[0].items():
        if value == value2:
            kiunguja_sets.append(idx)

In [193]:
validated[validated.index.isin(kiunguja_sets)].to_csv('sampla_data/eval_kiunguja.csv')

In [253]:
def dialect_evaluation_sets(df):
    dialect_set = []
    for idx, value in validated['sentence'].items():
        for idx2, value2 in df[0].items():
            if value == value2:
                dialect_set.append(idx)
            
    return validated[validated.index.isin(dialect_set)]

In [198]:
dialect_evaluation_sets(kibajuni).to_csv('sampla_data/eval_kibajuni.csv')
dialect_evaluation_sets(baratz).to_csv('sampla_data/eval_baratz.csv')
dialect_evaluation_sets(kimakunduchi).to_csv('sampla_data/eval_kimakunduchi.csv')
dialect_evaluation_sets(kimvita).to_csv('sampla_data/eval_kimvita.csv')
dialect_evaluation_sets(kipemba).to_csv('sampla_data/eval_kipemba.csv')
dialect_evaluation_sets(kitumbatu).to_csv('sampla_data/eval_kitumbatu.csv')

In [254]:
dialect_evaluation_sets(kibajuni)

,client_id,path,sentence,up_votes,down_votes,age,gender,accents,variant,locale,segment


In [256]:
validated

,client_id,path,sentence,up_votes,down_votes,age,gender,accents,variant,locale,segment
0,0133d8ddf5c1a3c678fde017e0b07d2835bfd707d5b3ec...,common_voice_sw_31428161.mp3,wachambuzi wa soka wanamtaja messi kama nyota ...,2,0,twenties,female,NaN,NaN,sw,NaN
1,01c95772efd3fbe4a1122206c7474c77ed6591c8c9fb00...,common_voice_sw_30317714.mp3,romario aliingia kwenye orodha ya wachezaji wa...,2,1,NaN,NaN,NaN,NaN,sw,NaN
2,023711185d4404ff398c2697f2e72868d1ecf69a92b581...,common_voice_sw_32116997.mp3,sote twesangaa twelipomuona mwalimu ali apika,2,1,twenties,male,NaN,NaN,sw,NaN
3,0244639ffd7ec755a01b21ea204735ca3c44443e9cf46c...,common_voice_sw_29002392.mp3,inajulikana kama shina la warangi,2,0,NaN,NaN,NaN,NaN,sw,NaN
4,04e78dc3038488a080fe3c76c28602d0db9e4eec2efbf0...,common_voice_sw_37088105.mp3,ni kikwazo kikubwa katika maendeleo ya afrika,8,5,NaN,NaN,NaN,NaN,sw,NaN
...,...,...,...,...,...,...,...,...,...,...,...
231137,457b3a2570720101c75d297cde767487e8f0a1a7f714cb...,common_voice_sw_30475794.mp3,hana pesa za kutosha,2,0,thirties,male,NaN,NaN,sw,NaN
231138,457b3a2570720101c75d297cde767487e8f0a1a7f714cb...,common_voice_sw_30475795.mp3,chakula chako si kitamu vile,2,1,thirties,male,NaN,NaN,sw,NaN
231139,457b3a2570720101c75d297cde767487e8f0a1a7f714cb...,common_voice_sw_30475797.mp3,aliitwa mara tatu,2,0,thirties,male,NaN,NaN,sw,NaN
231140,457b3a2570720101c75d297cde767487e8f0a1a7f714cb...,common_voice_sw_30475802.mp3,maradhi ya tumbo sio mchezo,2,1,thirties,male,NaN,NaN,sw,NaN
